# Density Based Clustering By Obaid Ullah GCS1947 

In [86]:
import pandas as pd
import numpy as np
import networkx as nx



In [87]:
def get_sortednodes(graph, weight_df):
    c = 0
    node_w_dict = {'node':[], 'w':[]}
    for node in graph.nodes():
        node_acc = 0
        c = c + 1
        #nborlist = nx.classes.function.all_neighbors(G,node)
        
        #for nbor in nborlist:
            
            #nbor_weight = G.get_edge_data(node,nbor)
            
            #this = weight_df.sort_values([node,'w'],ascending=False).groupby('w')
            #print(this)
            #weight_list = [v['w'].iloc[0] for k, v in this if k==node]
            #edge_weight = weight_df.loc[weight_df['u']==node, 'w']
            
            
        nodeW = get_node_weight(weight_df,node )
        
        node_acc = nodeW + node_acc
        #print(node_acc)
        node_w_dict['node'].append(node)
        node_w_dict['w'].append(round(node_acc,2))
        if c==2:
            break
        
        
    node_w_df = pd.DataFrame.from_dict(node_w_dict)

    return node_w_df.sort_values(by=['w'],ascending=False)




In [88]:
# clustering 


def get_edgew(graph):
    
    
    #print('This node',this_node, 'has these many neighbours: ',len(list(this_node_nbors)))
    #print()
    #cn_nbors = nx.classes.function.all_neighbors(cn,max_node[0])
    
    
    edge_weight = {'u':[], 'v':[], 'w':[]}
    
    for node in graph.nodes():
        this_node_nbors = nx.classes.function.all_neighbors(graph,node)
        for nbor in list(this_node_nbors):
            wt = 0
            
            wt = len(list(nx.common_neighbors(graph,node, nbor)))
            #print(node, nbor, wt)

            edge_weight['u'].append(node)
            edge_weight['v'].append(nbor)
            edge_weight['w'].append(wt)
        this_node_nbors = None
        
      
    edge_weight = pd.DataFrame.from_dict(edge_weight)
    return edge_weight
    #return edge_weight.sort_values(by=['w'],ascending=False)   
   

In [89]:
def get_node_weight(w_df, n):
    accm_w = 0 
    for idx, row in w_df.iterrows():
        if row['u']==n:
            #print('node ---',row['u'], 'weight ----- ',row['w'])
            accm_w = accm_w + row['w']
            
    return accm_w
   

In [90]:
def get_sum_nbor(candidate, Graph):
        
        accm_w = 0
        for nodes in Graph.nodes():
            nbors = len(list(nx.common_neighbors(G,candidate,node)))
            accm_w = accm_w + nbors
        return accm_w

In [91]:


def total_edges(cluster, graph, e_candidate):
    accm_edges = 0
    for node in cluster:
        edges = graph.number_of_edges(node, e_candidate)
        accm_edges = edges + accm_edges
       
    return accm_edges 


In [92]:
def find_v(cl_n, e_wt):
    for idx, rows in e_wt.iterrows():
        if cl_n == rows['u']:
            #print(idx)
            return e_wt.iloc[idx]

In [93]:

def cluster_add(cluster,Graph, cluster_nodes_sorted, edge_weights):
    
    c=0
    # start with sorted cluster nodes
    for idx, cl_node in cluster_nodes_sorted.iterrows():
        
        
        
        
        cl_node = cl_node['node']
        
        #get cluster node neighbors in GRAPH
        nborlist = nx.classes.function.all_neighbors(Graph,cl_node)

        edge_weight = {'u':[], 'v':[], 'w':[]}

        #for each nbor in cluster Node
        for nbor in nborlist:
            nbr_weight = 0
            #print(nbor)
            
            #chech if this neighbor is in the cluster or not
            if nbor not in cluster_nodes_sorted:
                #print('True')
                
                # for every node in the cluster nodes sorted DF
                for idx, row in edge_weights.iterrows():
                    #check for this nbor and cluster weights in the graph weights table edge_weights
                    if row['u'] == cl_node and row['v']== nbor:
                        c = c + 1
                        if c==2:
                            edge_weight = pd.DataFrame.from_dict(edge_weight)
                            edge_weight = edge_weight.sort_values(by=['w'],ascending=False)
                            return edge_weight
                        print(' row form graph',row['u'], row['v'])
                        print( 'loop cluster node ', cl_node,'nbor in loop: ',nbor)
                        nbr_weight = nbr_weight + row['w']

                        edge_weight['u'].append(cl_node)
                        edge_weight['v'].append(nbor)
                        edge_weight['w'].append(nbr_weight)
    edge_weight = pd.DataFrame.from_dict(edge_weight)
    edge_weight = edge_weight.sort_values(by=['w'],ascending=False)
    return edge_weight

In [99]:

def add_nbor_to_cluster(nbor_from_graph):
    u , v , w = edge_from_cluster.iloc[0]
    cluster.add_edges_from([(u,v,{'w':w})])


    density = nx.classes.function.density(cluster)
    number_of_nodes = nx.classes.function.number_of_nodes(cluster)


    print(number_of_nodes, density)
    edges_total = w
    cp = edges_total /(number_of_nodes * density)
    
    if cp<0.2 and density<0.6:
        cluster.remove_edges_from([(u,v,{'w':w})])
        cluster.remove_node(v)

def add_cluster_to_list(cluster, cluster_list):
    cluster_list.append(cluster)
    return cluster_list
    
    
    
    

In [97]:
##################------------------MAIN--------------------####################



def main_f():

    #Read graph into Pandas Dataframe
    df = pd.read_csv('file.csv', sep="\t", names=['u','v','w'])

    #read Graph from Pandas Dataframe into Netoworkx Graph

    G = nx.from_pandas_edgelist(df,'u','v','w')

    #density and cp threshold
    density = 0.6
    cp = 0.2

    #list of clusters from Graph
    cluster_list = []

    #Get edges weights of this graph
    # WARNING !!!!!!!!!!!!!!! TAKES A LONG TIME CHECK THIS FUNCTION CODE TO REMOVE BREAKS BEFORE REMOVING COMMENT
    edge_weights = get_edgew(G)
    #edge_weights


    # WARNING HERE !!!!!! TAKES A LONG TIME 
    sorted_nodes = get_sortednodes(G, edge_weights)
    #sorted_nodes


    #get nodes sorted by weight from Graph
    #sorted_node_df = get_sortednodes(,G)

    #MAKE A NEW CLUSTER 
    cluster = nx.Graph()

    #ADD FIRST TOP NODE TO CLUSTER ACCORDING TO SPECS IN ALGO
    #cl_node = sorted_nodes.iloc[0]['node']
    
    #for each of the top nodes in the Graph 
    # add them to cluster 
    i=0
    for idx, row in sorted_nodes.iterrows():
        if i==1:
            break
        cl_node = row['node']
        # add DESTINATION "V" OF THE FIRST NODE 
        cl_vw = find_v(cl_node, edge_weights)
        u,v,w = cl_vw[0],cl_vw[1],cl_vw[2]
        print(u,v,w)
        cluster.add_edges_from([(u,v,{'w':w})])

        # sort nodes in cluster and get their sorted edges and sorted top nodes 
        cluster_edge_weight = get_edgew(cluster)
        cluster_nodes_sorted  = get_sortednodes(cluster, cluster_edge_weight)
        #cluster.nodes()

        #GET NEIHBORS FROM GRAPH AND ADD THEM TO CLUSTER ACCORDING TO ALGO FLOW
        nbor_from_graph = cluster_add(cluster,G, cluster_nodes_sorted,edge_weights)
        #ADD THIS CLUSTER TO CLUSTER LIST
        cluster_list = add_cluster_to_list(cluster, cluster_list)
        #delete this cluster 
        cluster = None
        i=i+1
    
    return cluster_list

In [100]:
# RUN THIS ALGO


myclusters = []

myclusters = main_f()

TFC3 MYO4 208
 row form graph TFC3 MYO4
loop cluster node  TFC3 nbor in loop:  MYO4


In [103]:
myclusters[0].nodes()

NodeView(('TFC3', 'MYO4'))